In [2]:
pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 8.4 MB/s eta 0:00:0000:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.5/200.5 kB 5.8 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=656f5767213f82e8717568d010e085c00ba820b4f483af2125d4553e722f3ade
  Stored in directory: /home/codespace/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark
Note: you may need to restart the kernel to use updated packages.


In [3]:
# Import các module cần thiết
from pyspark.sql import SparkSession
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import RegressionEvaluator

In [4]:
# Khởi tạo một phiên Spark
spark = SparkSession.builder \
    .appName("LinearRegressionExample") \
    .getOrCreate()

24/05/14 03:56:15 WARN Utils: Your hostname, codespaces-429cb5 resolves to a loopback address: 127.0.0.1; using 172.16.5.4 instead (on interface eth0)
24/05/14 03:56:15 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/14 03:56:16 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
# Tạo dữ liệu giả định và chuyển đổi thành DataFrame PySpark
data = [(0.0, 1.0), (1.0, 2.0), (2.0, 3.0), (3.0, 4.0)]
df = spark.createDataFrame(data, ["label", "features"])
df.show()

+-----+--------+
|label|features|
+-----+--------+
|  0.0|     1.0|
|  1.0|     2.0|
|  2.0|     3.0|
|  3.0|     4.0|
+-----+--------+



In [6]:
# Tạo một đối tượng VectorAssembler để tạo vectơ đặc trưng từ cột "features"
assembler = VectorAssembler(
    inputCols=["features"],
    outputCol="features_vector"
)

In [7]:

# Chuyển đổi DataFrame để thêm cột features_vector
df = assembler.transform(df)

# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
train_data, test_data = df.randomSplit([0.8, 0.2], seed=1234)

In [8]:
# Khởi tạo mô hình Linear Regression
lr = LinearRegression(featuresCol="features_vector", labelCol="label")

24/05/14 03:56:35 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [9]:
# Huấn luyện mô hình trên tập huấn luyện
lr_model = lr.fit(train_data)

24/05/14 03:56:38 WARN Instrumentation: [ca8bb1f8] regParam is zero, which might cause numerical instability and overfitting.
24/05/14 03:56:39 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
24/05/14 03:56:39 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.VectorBLAS
24/05/14 03:56:39 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


In [10]:
# Dự đoán trên tập kiểm tra
predictions = lr_model.transform(test_data)

In [11]:
# Đánh giá mô hình bằng cách tính MSE trên tập kiểm tra
evaluator = RegressionEvaluator(labelCol="label", predictionCol="prediction", metricName="mse")
mse = evaluator.evaluate(predictions)
print("Mean Squared Error trên tập kiểm tra:", mse)

Mean Squared Error trên tập kiểm tra: 7.888609052210118e-31


In [12]:
# Hiển thị các hệ số của mô hình
print("Hệ số của mô hình:", lr_model.coefficients)

Hệ số của mô hình: [0.9999999999999987]


In [13]:
lr_model.write().overwrite().save("logistic_regression_model")

In [2]:
from pyspark.ml.classification import LogisticRegressionModel
from pyspark.sql import SparkSession

# Khởi tạo SparkSession
spark = SparkSession.builder \
    .appName("LoadModelExample") \
    .getOrCreate()

# Đọc lại mô hình từ tệp Parquet
lrModel = LogisticRegressionModel.load("logistic_regression_model")


IllegalArgumentException: requirement failed: Error loading metadata: Expected class name org.apache.spark.ml.classification.LogisticRegressionModel but found class name org.apache.spark.ml.regression.LinearRegressionModel